# Library

Instalisasi Library

In [ ]:
!pip install mplfinance
!pip install yfinance
!pip install backtesting
!pip install plotly
!pip install vectorbt
!pip install numpy
!pip install pandas
!pip install pandas-ta
!pip install matplotlib
!pip install scikit-learn
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00


# Data Gathering

Download Saham dalam 3 Waktu Sekaligus

In [ ]:
#Pengambiland Dataset Dari YFInance

import yfinance as yf

# Parameter
ticker = "ADRO.JK"
start_date1 = "2019-01-01"
end_date = "2024-01-01"
interval = "1d"

# Ambil data BYAN.JK dalam periode 5 tahun
data_saham_1= yf.download(ticker, start=start_date1, end=end_date, interval=interval, auto_adjust=False, multi_level_index=False)

# Tampilkan data
data_saham_1


# Data Pre-Processing

Implementasi Parabolic SAR

In [ ]:
import fileinput

file_path = "/usr/local/lib/python3.11/dist-packages/pandas_ta/momentum/squeeze_pro.py"

with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace("from numpy import NaN as npNaN", "from numpy import nan as npNaN"), end='')

#Implementasi Teknikal Indikator
import pandas_ta as ta
import pandas as pd


# Implementasi Parabolic SAR
psar = ta.psar(high=data_saham_1['High'], low=data_saham_1['Low'], close=data_saham_1['Close'], af=0.02, max_af=0.2)
data_saham_1 = pd.concat([data_saham_1, psar.add_suffix('_PSAR')], axis=1)


data_saham_1

Penerapan Look Up Day Close

In [ ]:
#Look up day close
#
lookup_day = 10

data_saham_1['lookup_day_close'] = data_saham_1['Close'].shift(-lookup_day)

data_saham_1

Penerapan Label Buy/Hold/Sell

In [ ]:
import pandas as pd
import numpy as np

# Target profit dan loss
target_profit = 0.08
target_loss = -0.04

# Fungsi untuk labeling
def apply_label(row):
    selisih = (row['lookup_day_close'] - row['Close']) / row['Close']
    if selisih > target_profit:
        return "Buy"
    elif selisih < target_loss:
        return "Sell"
    else:
        return "Hold"

# Terapkan ke data_saham_1
data_saham_1['label'] = data_saham_1.apply(apply_label, axis=1)

# Tampilkan data yang sudah dilabel
data_saham_1[['Close', 'lookup_day_close', 'label']]


Penerapan Fitur dan Filter PSAR

In [ ]:
import pandas as pd

#  Filter: Sinyal perpindahan titik PSAR
def psar_shift_signal(row):
    if row['psar_trend'] == 1 and row['psar_trend_prev'] == -1:
        return 1  # Perpindahan ke atas (Bearish reversal)
    elif row['psar_trend'] == -1 and row['psar_trend_prev'] == 1:
        return -1  # Perpindahan ke bawah (Bullish reversal)
    else:
        return 0  # Tidak ada perubahan

#  Hitung trend PSAR
def detect_trend(row):
    psarl = row['PSARl_0.02_0.2_PSAR']
    psars = row['PSARs_0.02_0.2_PSAR']

    if not pd.isna(psarl) and pd.isna(psars):
        return 1  # Tren naik (Bullish)
    elif pd.isna(psarl) and not pd.isna(psars):
        return -1  # Tren turun (Bearish)
    else:
        return 0  # Tidak berubah

#  Gabungkan PSAR atas dan bawah
def gabungkan_psar(row):
    psarl = row['PSARl_0.02_0.2_PSAR']
    psars = row['PSARs_0.02_0.2_PSAR']

    if pd.isna(psarl):
        return psars
    elif pd.isna(psars):
        return psarl
    else:
        return psarl

#  Mulai proses pada data_saham_1
df = data_saham_1.copy()  # Hindari SettingWithCopyWarning

# Hapus kolom duplikat kalau ada
df = df.loc[:, ~df.columns.duplicated()]

# Proses PSAR
df['psar_trend'] = df.apply(detect_trend, axis=1)
df['psar_trend_prev'] = df['psar_trend'].shift(1)
df['psar_shift'] = df.apply(psar_shift_signal, axis=1)
df['PSAR'] = df.apply(gabungkan_psar, axis=1)
df['d1'] = df['Close'] - df['PSAR']
df['d2'] = df['d1'] - df['d1'].shift(1)

# Jika terjadi perpindahan titik PSAR, samakan nilai d2 dengan sebelumnya
for j in range(1, len(df)):
    if df.loc[df.index[j], 'psar_shift'] != 0:
        df.loc[df.index[j], 'd2'] = df.loc[df.index[j-1], 'd2']

# Simpan kembali ke variabel utama
data_saham_1 = df

# Cek hasil
data_saham_1


Drop Nilai NaN dalam dataset

In [ ]:
df = data_saham_1

df_final = df[['Close', 'd1', 'd2', 'psar_shift']]
target = df['label']

df_final_clean = df_final.dropna()
target_clean = target.loc[df_final_clean.index]

data_saham_final_1 = df_final_clean
target_1 = target_clean

print(type(data_saham_final_1))

data_saham_final_1


Pembagian Dataset dan Balacing Data menggunakan SMOTE

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  Dataset dan target yang sudah diproses (data_saham_final_1 harus sudah siap sebelumnya)
data = data_saham_final_1  # Data fitur
target = target_1          # Label target
data.index = pd.to_datetime(data.index)

#  Definisikan rolling windows
windows = [
    ("2019-01-01", "2022-01-01"),
    ("2020-01-01", "2023-01-01"),
    ("2021-01-01", "2023-06-01"),
]

X_trains, X_tests, y_trains, y_tests = [], [], [], []

#  Simpan label untuk plotting
labels_before, labels_after = [], []

#  Loop untuk rolling windows
for i, (start_train, end_train) in enumerate(windows):
    # Train dan test berdasarkan tanggal
    X_train = data.loc[start_train:end_train]
    y_train = target.loc[start_train:end_train]
    X_test = data.loc[end_train:"2024-01-01"]
    y_test = target.loc[end_train:"2024-01-01"]

    # Simpan label sebelum SMOTE untuk plot
    labels_before.append(y_train)

    # SMOTE
    smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=2)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Simpan label setelah SMOTE untuk plot
    labels_after.append(y_train_resampled)

    # Simpan hasil
    X_trains.append(X_train_resampled)
    y_trains.append(y_train_resampled)
    X_tests.append(X_test)
    y_tests.append(y_test)

#  Visualisasi distribusi sebelum dan sesudah SMOTE
df_labels = pd.DataFrame({
    'Dataset': sum([[f'Rolling {i+1}'] * len(y) for i, y in enumerate(labels_before)], []),
    'Label': pd.concat(labels_before)
})

df_labels_after = pd.DataFrame({
    'Dataset': sum([[f'Rolling {i+1}'] * len(y) for i, y in enumerate(labels_after)], []),
    'Label': pd.concat(labels_after)
})

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sebelum SMOTE
sns.countplot(y='Label', hue='Dataset', data=df_labels, palette='Set2', ax=axes[0])
axes[0].set_title("Distribusi Label Sebelum SMOTE")
axes[0].set_xlabel("Jumlah")
axes[0].set_ylabel("Kategori")
axes[0].legend(title="Dataset")
axes[0].spines[['top', 'right']].set_visible(False)

# Setelah SMOTE
sns.countplot(y='Label', hue='Dataset', data=df_labels_after, palette='Set2', ax=axes[1])
axes[1].set_title("Distribusi Label Setelah SMOTE")
axes[1].set_xlabel("Jumlah")
axes[1].set_ylabel("")
axes[1].legend(title="Dataset")
axes[1].spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.show()


Normalisasi menggunakan Standar Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

#  Gunakan hasil dari proses SMOTE sebelumnya
# (X_trains, X_tests, y_trains, y_tests sudah tersedia dari kode pertama)

#  Buat nama-nama dataset untuk logging
dataset_names = [f"Rolling {i+1}" for i in range(len(X_trains))]

#  Simpan hasil standardisasi
X_train_scaled_list, X_test_scaled_list = [], []

#  Loop untuk setiap dataset rolling
for X_train_resampled, X_test, name in zip(X_trains, X_tests, dataset_names):
    print(f"🔹 Proses standardisasi untuk {name}...")

    # Inisialisasi StandardScaler
    scaler = StandardScaler()

    # Standardisasi pada data training (fit + transform)
    X_train_scaled = scaler.fit_transform(X_train_resampled)

    # Transformasi pada data testing (transform saja)
    X_test_scaled = scaler.transform(X_test)

    # Simpan hasil
    X_train_scaled_list.append(X_train_scaled)
    X_test_scaled_list.append(X_test_scaled)

    print(f"✅ Selesai untuk {name}\n")

#  Assign ke variabel terpisah agar mudah digunakan
X_train_1_scaled, X_train_2_scaled, X_train_3_scaled = X_train_scaled_list
X_test_1_scaled, X_test_2_scaled, X_test_3_scaled = X_test_scaled_list

#  Assign juga untuk y_train dari hasil SMOTE
y_train_1_resampled, y_train_2_resampled, y_train_3_resampled = y_trains
y_test_1, y_test_2, y_test_3 = y_tests

#  Output hasil
print("✅ Semua dataset selesai diproses dengan standardisasi StandardScaler.")


# SVM Model

Penggunaan Grid Search untuk mencari nilai parameter terbaik pada setiap model

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

#  Definisi parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'gamma': [1, 0.5, 0.1, 0.05],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

#  Inisialisasi variabel model kosong
svm_model_1 = None
svm_model_2 = None
svm_model_3 = None

#  Simpan juga hasil terbaik
best_params_dict = {}

#  Loop untuk masing-masing rolling window
for i, (X_train_scaled, y_train_resampled, name) in enumerate(zip(X_train_scaled_list, y_trains, dataset_names), start=1):
    print(f" Training SVM untuk {name} (rolling window {i})...")

    svm_model = GridSearchCV(
        estimator=SVC(),
        param_grid=param_grid,
        refit=True,
        verbose=1
    )

    svm_model.fit(X_train_scaled, y_train_resampled)

    # Simpan ke variabel spesifik per rolling window
    if i == 1:
        svm_model_1 = svm_model
    elif i == 2:
        svm_model_2 = svm_model
    elif i == 3:
        svm_model_3 = svm_model

    # Simpan hasil terbaik ke dictionary
    model_key = f"data_saham_{i}"
    best_params_dict[model_key] = {
        "Best Parameters": svm_model.best_params_,
        "Best Estimator": svm_model.best_estimator_,
        "Best Score": svm_model.best_score_
    }

    print(f"✅ Model untuk {model_key} selesai.\n")

# 🔹 Menampilkan hasil terbaik
print(" Hasil GridSearchCV (Model yang Berbeda per Rolling):")
for key, best_params in best_params_dict.items():
    print(f" {key}")
    print("Best Parameters:", best_params["Best Parameters"])
    print("Best Estimator:", best_params["Best Estimator"])
    print("Best Score:", best_params["Best Score"])
    print("=" * 50)


Menyimpan Nilai Parameter kedalam data Saham

In [ ]:
#  Dictionary hasil terbaik dengan struktur per rolling
best_params_dict = {
    "data_saham_1": {
        "Best Parameters": svm_model_1.best_params_,
        "Best Estimator": svm_model_1.best_estimator_,
        "Best Score": svm_model_1.best_score_
    },
    "data_saham_2": {
        "Best Parameters": svm_model_2.best_params_,
        "Best Estimator": svm_model_2.best_estimator_,
        "Best Score": svm_model_2.best_score_
    },
    "data_saham_3": {
        "Best Parameters": svm_model_3.best_params_,
        "Best Estimator": svm_model_3.best_estimator_,
        "Best Score": svm_model_3.best_score_
    }
}


# Model Evaluation

In [ ]:
import seaborn as sns

sns.heatmap(data_saham_final_1)

Evaluasi Model memakai Accuracy, Classification Report, dan Confusion Matrix

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#  List dataset yang akan diuji
datasets_test = [X_test_1_scaled, X_test_2_scaled, X_test_3_scaled]
targets_test = [y_test_1, y_test_2, y_test_3]
dataset_names = ['data_saham_1', 'data_saham_2', 'data_saham_3']

#  Evaluasi Model SVM untuk setiap dataset
for i, (X_test, y_test, name) in enumerate(zip(datasets_test, targets_test, dataset_names)):
    print(f" Evaluasi Model untuk {name}...\n")

    # Prediksi hasil menggunakan model terbaik dari GridSearchCV
    best_model = best_params_dict[name]["Best Estimator"]
    y_pred = best_model.predict(X_test)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=['Buy', 'Hold', 'Sell'],
                yticklabels=['Buy', 'Hold', 'Sell'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {name}')
    plt.show()

    # Accuracy & Classification Report
    accuracy = accuracy_score(y_test, y_pred)
    print(f" Accuracy for {name}: {accuracy:.4f}\n")
    print(f" Classification Report for {name}:\n")
    print(classification_report(y_test, y_pred))
    print("=" * 50)


# Backtesting

Pengetesan Backtesting menggunakan model 1, 2, dan 3

In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy

# Download Data Saham
data = yf.download("ADRO.JK", start="2024-01-01", end="2024-12-31", auto_adjust=False, multi_level_index=False)

# Pastikan hanya mengambil kolom yang diperlukan
data_f = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data_f.dropna()

# Implementasi Parabolic SAR0
psar = ta.psar(high=data_f['High'], low=data_f['Low'], close=data_f['Close'], af=0.02, max_af=0.2)
data_f = pd.concat([data_f, psar.add_suffix('_PSAR')], axis=1)
# Fungsi mendeteksi tren PSAR
def detect_trend(row):
    if not pd.isna(row['PSARl_0.02_0.2_PSAR']) and pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return 1  # Tren naik (Bullish)
    elif pd.isna(row['PSARl_0.02_0.2_PSAR']) and not pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return -1  # Tren turun (Bearish)
    else:
        return 0  # Tidak berubah

# Tambahkan kolom tren PSAR
data_f['psar_trend'] = data_f.apply(detect_trend, axis=1)
data_f['psar_trend_prev'] = data_f['psar_trend'].shift(1)
data_f['psar_shift'] = np.where((data_f['psar_trend'] != data_f['psar_trend_prev']), data_f['psar_trend'], 0)

data_f['PSAR'] = data_f[['PSARl_0.02_0.2_PSAR', 'PSARs_0.02_0.2_PSAR']].bfill(axis=1).iloc[:, 0]

data_f['d1'] = data_f['Close'] - data_f['PSAR']
data_f['d2'] = data_f['d1'].diff()

data_f = data_f[['Close', 'd1', 'd2', 'psar_shift']].dropna()

# Standardisasi dengan sigmoid
# def sigmoid_standardization(data):
#     return 1 / (1 + np.exp(-data))

# Standardisasi dengan StandardScaler
scaler = StandardScaler()
X_scaled_df = pd.DataFrame(scaler.fit_transform(data_f), columns=data_f.columns, index=data_f.index)

data = data.loc[data_f.index]

# Menggunakan model SVM yang telah dilatih sebelumnya
svm_models = [svm_model_1, svm_model_2, svm_model_3]

data['Signal_1'] = svm_model_1.predict(X_scaled_df)
data['Signal_2'] = svm_model_2.predict(X_scaled_df)
data['Signal_3'] = svm_model_3.predict(X_scaled_df)

map_signal = {1: "Buy", 0: "Hold", -1: "Sell"}

# # Strategi Backtest
def run_backtest(signal_column):
    class TradeStrategy(Strategy):
        def init(self):
            pass

        def next(self):
            i = len(self.data) - 1
            current_signal = self.data.df[signal_column].iloc[i]
            current_price = self.data.Close[-1]
            # 0.97 = 3%, 1,06 = 6% | 0.95 = 5% , 1,10 = 10%
            stop_loss_price = current_price * 0.92
            take_profit_price = current_price * 1.04
            if current_signal == "Buy" and not self.position.is_long:
              self.buy()
              # self.buy(sl=stop_loss_price, tp=take_profit_price)
              # self.buy(sl=stop_loss_price)
            elif current_signal == "Sell" and self.position.is_long:
              self.position.close()

    # Pastikan indentasi ini sejajar dengan `class FixedTradeStrategy`
    data_bt = data[['Open', 'High', 'Low', 'Close', 'Volume', signal_column]]
    bt = Backtest(data_bt, TradeStrategy, cash=1_000_000, commission=0.0002, exclusive_orders=True)
    return bt.run(), bt

# Jalankan Backtest untuk masing-masing model
results_1, bt_1 = run_backtest('Signal_1')
results_2, bt_2 = run_backtest('Signal_2')
results_3, bt_3 = run_backtest('Signal_3')

print("\n Hasil Backtest untuk Model 1:")
print(results_1)
bt_1.plot()

print("\n Hasil Backtest untuk Model 2:")
print(results_2)
bt_2.plot()

print("\n Hasil Backtest untuk Model 3:")
print(results_3)
bt_3.plot()

Backtesting dengan Stoploss

In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy

# Download Data Saham
data = yf.download("ADRO.JK", start="2024-01-01", end="2024-12-31", auto_adjust=False, multi_level_index=False)

# Pastikan hanya mengambil kolom yang diperlukan
data_f = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data_f.dropna()

# Implementasi Parabolic SAR0
psar = ta.psar(high=data_f['High'], low=data_f['Low'], close=data_f['Close'], af=0.02, max_af=0.2)
data_f = pd.concat([data_f, psar.add_suffix('_PSAR')], axis=1)
# Fungsi mendeteksi tren PSAR
def detect_trend(row):
    if not pd.isna(row['PSARl_0.02_0.2_PSAR']) and pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return 1  # Tren naik (Bullish)
    elif pd.isna(row['PSARl_0.02_0.2_PSAR']) and not pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return -1  # Tren turun (Bearish)
    else:
        return 0  # Tidak berubah

# Tambahkan kolom tren PSAR
data_f['psar_trend'] = data_f.apply(detect_trend, axis=1)
data_f['psar_trend_prev'] = data_f['psar_trend'].shift(1)
data_f['psar_shift'] = np.where((data_f['psar_trend'] != data_f['psar_trend_prev']), data_f['psar_trend'], 0)

data_f['PSAR'] = data_f[['PSARl_0.02_0.2_PSAR', 'PSARs_0.02_0.2_PSAR']].bfill(axis=1).iloc[:, 0]

data_f['d1'] = data_f['Close'] - data_f['PSAR']
data_f['d2'] = data_f['d1'].diff()

data_f = data_f[['Close', 'd1', 'd2', 'psar_shift']].dropna()

# Standardisasi dengan sigmoid
# def sigmoid_standardization(data):
#     return 1 / (1 + np.exp(-data))

# Standardisasi dengan StandardScaler
scaler = StandardScaler()
X_scaled_df = pd.DataFrame(scaler.fit_transform(data_f), columns=data_f.columns, index=data_f.index)

data = data.loc[data_f.index]

# Menggunakan model SVM yang telah dilatih sebelumnya
svm_models = [svm_model_1, svm_model_2, svm_model_3]

data['Signal_1'] = svm_model_1.predict(X_scaled_df)
data['Signal_2'] = svm_model_2.predict(X_scaled_df)
data['Signal_3'] = svm_model_3.predict(X_scaled_df)

map_signal = {1: "Buy", 0: "Hold", -1: "Sell"}

# # Strategi Backtest
def run_backtest(signal_column):
    class TradeStrategy(Strategy):
        def init(self):
            pass

        def next(self):
            i = len(self.data) - 1
            current_signal = self.data.df[signal_column].iloc[i]
            current_price = self.data.Close[-1]
            # 0.97 = 3%, 1,06 = 6% | 0.95 = 5% , 1,10 = 10%
            stop_loss_price = current_price * 0.92
            take_profit_price = current_price * 1.04
            if current_signal == "Buy" and not self.position.is_long:
              # self.buy()
              # self.buy(sl=stop_loss_price, tp=take_profit_price)
              self.buy(sl=stop_loss_price)
            elif current_signal == "Sell" and self.position.is_long:
              self.position.close()

    # Pastikan indentasi ini sejajar dengan `class FixedTradeStrategy`
    data_bt = data[['Open', 'High', 'Low', 'Close', 'Volume', signal_column]]
    bt = Backtest(data_bt, TradeStrategy, cash=1_000_000, commission=0.0002, exclusive_orders=True)
    return bt.run(), bt

# Jalankan Backtest untuk masing-masing model
results_1, bt_1 = run_backtest('Signal_1')
results_2, bt_2 = run_backtest('Signal_2')
results_3, bt_3 = run_backtest('Signal_3')

print("\n Hasil Backtest untuk Model 1:")
print(results_1)
bt_1.plot()

print("\n Hasil Backtest untuk Model 2:")
print(results_2)
bt_2.plot()

print("\n Hasil Backtest untuk Model 3:")
print(results_3)
bt_3.plot()

Backtesting dengan takeprofit dan stoploss

In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy

# Download Data Saham
data = yf.download("ADRO.JK", start="2024-01-01", end="2024-12-31", auto_adjust=False, multi_level_index=False)

# Pastikan hanya mengambil kolom yang diperlukan
data_f = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data_f.dropna()

# Implementasi Parabolic SAR0
psar = ta.psar(high=data_f['High'], low=data_f['Low'], close=data_f['Close'], af=0.02, max_af=0.2)
data_f = pd.concat([data_f, psar.add_suffix('_PSAR')], axis=1)
# Fungsi mendeteksi tren PSAR
def detect_trend(row):
    if not pd.isna(row['PSARl_0.02_0.2_PSAR']) and pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return 1  # Tren naik (Bullish)
    elif pd.isna(row['PSARl_0.02_0.2_PSAR']) and not pd.isna(row['PSARs_0.02_0.2_PSAR']):
        return -1  # Tren turun (Bearish)
    else:
        return 0  # Tidak berubah

# Tambahkan kolom tren PSAR
data_f['psar_trend'] = data_f.apply(detect_trend, axis=1)
data_f['psar_trend_prev'] = data_f['psar_trend'].shift(1)
data_f['psar_shift'] = np.where((data_f['psar_trend'] != data_f['psar_trend_prev']), data_f['psar_trend'], 0)

data_f['PSAR'] = data_f[['PSARl_0.02_0.2_PSAR', 'PSARs_0.02_0.2_PSAR']].bfill(axis=1).iloc[:, 0]

data_f['d1'] = data_f['Close'] - data_f['PSAR']
data_f['d2'] = data_f['d1'].diff()

data_f = data_f[['Close', 'd1', 'd2', 'psar_shift']].dropna()

# Standardisasi dengan sigmoid
# def sigmoid_standardization(data):
#     return 1 / (1 + np.exp(-data))

# Standardisasi dengan StandardScaler
scaler = StandardScaler()
X_scaled_df = pd.DataFrame(scaler.fit_transform(data_f), columns=data_f.columns, index=data_f.index)

data = data.loc[data_f.index]

# Menggunakan model SVM yang telah dilatih sebelumnya
svm_models = [svm_model_1, svm_model_2, svm_model_3]

data['Signal_1'] = svm_model_1.predict(X_scaled_df)
data['Signal_2'] = svm_model_2.predict(X_scaled_df)
data['Signal_3'] = svm_model_3.predict(X_scaled_df)

map_signal = {1: "Buy", 0: "Hold", -1: "Sell"}

# # Strategi Backtest
def run_backtest(signal_column):
    class TradeStrategy(Strategy):
        def init(self):
            pass

        def next(self):
            i = len(self.data) - 1
            current_signal = self.data.df[signal_column].iloc[i]
            current_price = self.data.Close[-1]
            # 0.97 = 3%, 1,06 = 6% | 0.95 = 5% , 1,10 = 10%
            stop_loss_price = current_price * 0.92
            take_profit_price = current_price * 1.04
            if current_signal == "Buy" and not self.position.is_long:
              # self.buy()
              self.buy(sl=stop_loss_price, tp=take_profit_price)
              # self.buy(sl=stop_loss_price)
            elif current_signal == "Sell" and self.position.is_long:
              self.position.close()

    # Pastikan indentasi ini sejajar dengan `class FixedTradeStrategy`
    data_bt = data[['Open', 'High', 'Low', 'Close', 'Volume', signal_column]]
    bt = Backtest(data_bt, TradeStrategy, cash=1_000_000, commission=0.0002, exclusive_orders=True)
    return bt.run(), bt

# Jalankan Backtest untuk masing-masing model
results_1, bt_1 = run_backtest('Signal_1')
results_2, bt_2 = run_backtest('Signal_2')
results_3, bt_3 = run_backtest('Signal_3')

print("\n Hasil Backtest untuk Model 1:")
print(results_1)
bt_1.plot()

print("\n Hasil Backtest untuk Model 2:")
print(results_2)
bt_2.plot()

print("\n Hasil Backtest untuk Model 3:")
print(results_3)
bt_3.plot()

In [ ]:
results_1["_trades"]

Melihat dalam bentuk Candle

In [ ]:
import mplfinance as mpf

# Ambil posisi buy/sell dari hasil backtest
trades = results_1['_trades']
# Konversi buy/sell dates ke datetime
buy_dates = pd.to_datetime(trades.loc[trades['Size'] > 0, 'EntryTime'])
sell_dates = pd.to_datetime(trades.loc[trades['Size'] < 0, 'ExitTime'])

# Pastikan semua tanggal ada di index data
buy_prices = data.loc[data.index.isin(buy_dates), 'Close']
sell_prices = data.loc[data.index.isin(sell_dates), 'Close']


# Pastikan indeks datetime dan urutan benar
ohlc = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
ohlc.index = pd.to_datetime(ohlc.index)
ohlc = ohlc.sort_index()

# Buat tanda pada grafik
apds = []

# Plot tanda Buy (↑)
if not buy_prices.empty:
    buy_plot = pd.Series(np.nan, index=ohlc.index)
    buy_plot[buy_prices.index] = buy_prices.values
    apds.append(
        mpf.make_addplot(buy_plot, type='scatter',
                         markersize=100, marker='^', color='lime')
    )

# Plot tanda Sell (↓)
if not sell_prices.empty:
    sell_plot = pd.Series(np.nan, index=ohlc.index)
    sell_plot[sell_prices.index] = sell_prices.values
    apds.append(
        mpf.make_addplot(sell_plot, type='scatter',
                         markersize=100, marker='v', color='red')
    )



# Plot chart
mpf.plot(ohlc, type='candle', style='yahoo',
         title='Sinyal Buy/Sell dari Backtest',
         volume=True, addplot=apds, figratio=(16, 9))



In [ ]:
import mplfinance as mpf

# Ambil posisi buy/sell dari hasil backtest
trades = results_2['_trades']
# Konversi buy/sell dates ke datetime
buy_dates = pd.to_datetime(trades.loc[trades['Size'] > 0, 'EntryTime'])
sell_dates = pd.to_datetime(trades.loc[trades['Size'] < 0, 'ExitTime'])

# Pastikan semua tanggal ada di index data
buy_prices = data.loc[data.index.isin(buy_dates), 'Close']
sell_prices = data.loc[data.index.isin(sell_dates), 'Close']


# Pastikan indeks datetime dan urutan benar
ohlc = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
ohlc.index = pd.to_datetime(ohlc.index)
ohlc = ohlc.sort_index()

# Buat tanda pada grafik
apds = []

# Plot tanda Buy (↑)
if not buy_prices.empty:
    buy_plot = pd.Series(np.nan, index=ohlc.index)
    buy_plot[buy_prices.index] = buy_prices.values
    apds.append(
        mpf.make_addplot(buy_plot, type='scatter',
                         markersize=100, marker='^', color='lime')
    )

# Plot tanda Sell (↓)
if not sell_prices.empty:
    sell_plot = pd.Series(np.nan, index=ohlc.index)
    sell_plot[sell_prices.index] = sell_prices.values
    apds.append(
        mpf.make_addplot(sell_plot, type='scatter',
                         markersize=100, marker='v', color='red')
    )



# Plot chart
mpf.plot(ohlc, type='candle', style='yahoo',
         title='Sinyal Buy/Sell dari Backtest',
         volume=True, addplot=apds, figratio=(16, 9))



In [ ]:
import mplfinance as mpf

# Ambil posisi buy/sell dari hasil backtest
trades = results_3['_trades']
# Konversi buy/sell dates ke datetime
buy_dates = pd.to_datetime(trades.loc[trades['Size'] > 0, 'EntryTime'])
sell_dates = pd.to_datetime(trades.loc[trades['Size'] < 0, 'ExitTime'])

# Pastikan semua tanggal ada di index data
buy_prices = data.loc[data.index.isin(buy_dates), 'Close']
sell_prices = data.loc[data.index.isin(sell_dates), 'Close']


# Pastikan indeks datetime dan urutan benar
ohlc = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
ohlc.index = pd.to_datetime(ohlc.index)
ohlc = ohlc.sort_index()

# Buat tanda pada grafik
apds = []

# Plot tanda Buy (↑)
if not buy_prices.empty:
    buy_plot = pd.Series(np.nan, index=ohlc.index)
    buy_plot[buy_prices.index] = buy_prices.values
    apds.append(
        mpf.make_addplot(buy_plot, type='scatter',
                         markersize=100, marker='^', color='lime')
    )

# Plot tanda Sell (↓)
if not sell_prices.empty:
    sell_plot = pd.Series(np.nan, index=ohlc.index)
    sell_plot[sell_prices.index] = sell_prices.values
    apds.append(
        mpf.make_addplot(sell_plot, type='scatter',
                         markersize=100, marker='v', color='red')
    )



# Plot chart
mpf.plot(ohlc, type='candle', style='yahoo',
         title='Sinyal Buy/Sell dari Backtest',
         volume=True, addplot=apds, figratio=(16, 9))

